<a href="https://colab.research.google.com/github/rhnobre/DataScience/blob/main/XGBoost%20-%20aprenda%20este%20algoritmo%20de%20Machine%20Learning%20em%20Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# O Que é o Algoritmo XGBoost?

---

XGBoost é um dos algoritmos mais utilizados por cientistas de dados, apresentando resultados superiores principalmente em problemas de previsão envolvendo dados estruturados/tabulares.

Dados tabulares são aquele tipo de estrutura usado pelo Pandas, os conhecidos DataFrames. Por suas características, consegue lidar eficientemente (e com robustez) com uma grande variedade de tipos diferentes de dados.

Se você está lidando com problemas de regressão, pontuação ou classificação, definitivamente você precisa aprender a usar o XGBoost.

Neste artigo, vou falar mais sobre esse algoritmo de *Machine Learning* que vem dominando cada vez mais as competições do [Kaggle](http://www.kaggle.com).

## Uma introdução ao XGBoost

O nome XGBoost vem de *e**X**treme **G**radient **Boost**ing*, e representa uma categoria de algoritmo baseada em *Decision Trees* (árvores de decisão) com **Gradient Boosting** (aumento de gradiente).

Aumento de gradiente significa que o algoritmo usa o algoritmmo *Gradient Descent* para minimizar a perda (*loss*) enquanto novos modelos vão sendo adicionado.

Extremamente flexível - uma vez é possui um grande número de hiperparâmetros passíveis de aperfeiçoamento -, você consegue ajustar adequadamente o XGBoost para o cenário do seu problema, seja ele qual for.

### Árvores de Decisão e *Gradient Boosting*

Árvores de Decisão são métodos onde existe uma função que recebe um vetor de valores (de atributos) como entrada e retorna uma decisão (de saída).

Para uma árvore de decisão chegar no valor de saída, executa uma série de etapas, ou testes, criando várias ramificações ao longo do processo.

Cada nó dessa árvore representa uma decisão única. Quanto mais vezes um atributo for usado para as tomadas de decisão, maior será sua importância relativa no modelo.

### *Gradient Boosting*

*Gradient Boosting* é uma técnica relativamente recente, e que se mostrou muito poderosa.

Tamanho é o seu potencial, que algoritmos baseados nessa técnica têm ganhando cada vez mais destaque em projetos de Data Science e competições do Kaggle.

O princípio do *Gradient Boosting* é a capacidade de combinar resultados de muitos classificadores "fracos", tipicamente árvores de decisão, que se combinam para formar algo parecido com um "comitê forte de decisão".

## Implementando XGBoost com Python

Para mostrar na prática como se implementa o XGBoost, vou usar a API do Kaggle para baixar o *dataset* [House Prices: Advanced Regression Techniques](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data).

Se você não conhece a [API Kaggle](https://github.com/Kaggle/kaggle-api) ou não quer usar ela, não tem problema! É só baixar o arquivo *zip* diretamente do link acima.

Como a ideia é apenas mostrar a implementação da técnica, nem irei baixar todo o conjunto de dados, apenas um único arquivo: **train.csv**.



In [ ]:
!mkdir -p data
!kaggle competitions download -c house-prices-advanced-regression-techniques -f train.csv -p data

### Importando e preparando os dados para o Pandas

Após importar o pacote Pandas, vou importar aquele arquivo csv para dentro de um DataFrame.

Para você entender o *dataset*, vou listar as 5 primeiras linhas de entrada do modelo. Recomendo que você baixe o *notebook* na tua máquina e faça o mesmo.

Se você estiver executando diretamente na sua máquina, pode ser que você não tenha instalado o pacote ```xgboost```. Nesse caso, você pode instalar pela linha de comando:

``` pip install xgboost```

In [ ]:
# importar bibliotecas necessárias
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

# importar train.csv em DataFrame
df = pd.read_csv('data/train.csv')

# visualizar as 5 primeiras entradas
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


Neste *post* nós não vamos fazer uma análise aprofundada. O objetivo é que você aprenda a implementar o XGBoost nos seus modelos.

Por isso, já dei uma olhada prévia no modelo e identifiquei que nossa variável alvo é a coluna ```SalePrice```. Também já verifiquei que não existem valores nulos na mesma.

Para não termos que trabalhar as variáveis categóricas aqui, vou eliminar todas as colunas do tipo ```object``` e ficar apenas com as variáveis numéricas.

Na sequência, vou separar entre o DataFrame entre as variáveis X e y e dividir o *dataset* entre conjuntos de treino e teste.

Por último, vou usar a classe ```SimpleImputer()``` para lidar rapidamente com os valores ausentes do *dataset*.

In [ ]:
# separar entre as variáveis X e y
y = df['SalePrice']
X = df.drop(['SalePrice'], axis=1).select_dtypes(exclude=['object'])

# dividir entre conjuntos de treino e teste
train_X, test_X, train_y, test_y = train_test_split(X.values, y.values, test_size=0.2)

# lidar com os valores ausentes
df_imputer = SimpleImputer()
train_X = df_imputer.fit_transform(train_X)
test_X = df_imputer.transform(test_X)

### Implementando um modelo de XGBoost com Python

Com os nossos dados já preparados, agora é a hora de construir um modelo de *Machine Learning* XGBoost.

Seguindo o mesmo padrão daquilo que você já está acostumado a fazer com o ```sklearn```, depois de instanciar ```XGBRegressor()``` basta executar o método ```fit()```, passando o *dataset* de treino como argumento.

Na sequência, vamos realizar as previsões e calcular o erro médio absoluto para ver o desempenho do modelo.

In [ ]:
# instanciar o modelo XGBoost
model = XGBRegressor()

# chamar o fit para o modelo
model.fit(train_X, train_y, verbose=False)

# fazer previsões em cima do dataset de teste
predictions = model.predict(test_X)

print("Erro Médio Absoluto: {:.2f}".format(mean_absolute_error(predictions, test_y)))

[05:08:55] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Erro Médio Absoluto: 15972.56


Pronto. Construir um modelo é relativamente rápido comparado às outras etapas de um projeto.

Entretanto, tem um passo a mais que faz muita diferença nos seus resultados usando esse algoritmo.

### Parameter Tunning: ajustando parâmetros do XGBoost


De maneira simples e direta, você viu na etapa anterior como construir um modelo XGBoost. Mas será que é tão simples assim conseguir aqueles resultados incríveis assim nas competições do Kaggle?

Com certeza não! Se fosse assim, qualquer pessoa poderia estar no topo do *ranking*.

O primeiro ponto que eu queria ressaltar é que não fizemos nenhuma análise aprofundada durante a etapa exploratória dos nossos dados - a fase mais importante de qualquer projeto de Data Science.

O segundo ponto é que o XGBoost é extremamente robusto e poderoso, mas tem um grande número de parâmetros para serem ajustados ao problema.

Para ajustar esses parâmetros, não tem escapatória. Você tem que entender a teoria por trás do XGBoost, seus conceitos matemáticos e estatísticos.

##  Evoluindo com a prática

Não existem atalhos em Data Science. Você tem que criar e participar do máximo de projetos que puder. 

Não seja uma pessoa que primeiro estuda tudo do assunto para no final começar a escrever códigos.

Muita coisa você vai pegando com a experiência, então minha recomendação é:

* Começe na prática, replicando métodos e *notebooks* dos outros;
* Aprenda a teoria conforme for evoluindo, de acordo com a necessidade

